In [2]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

sys.path.append(os.getenv("ROOT"))

In [3]:
import polars as pl
import numpy as np
from datetime import date
import silverfund.data_access_layer as dal
from silverfund.enums import Interval
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Load dataset
start_date = date(1963, 1, 1)
end_date = date(2012, 12, 31)

df = dal.load_crsp(interval=Interval.DAILY, start_date=start_date, end_date=end_date)

df

Loading CRSP Daily: 100%|██████████| 50/50 [00:02<00:00, 17.11it/s]


permno,permco,date,ncusip,ticker,shrcd,exchcd,siccd,prc,ret,retx,vol,shrout,cfacshr
i64,i64,date,str,str,i64,i64,i64,f64,f64,f64,f64,f64,f64
10000,7952,1986-01-07,"""68391610""","""OMFGA""",10,3,3990,-2.5625,null,null,1000.0,3680.0,1.0
10000,7952,1986-01-08,"""68391610""","""OMFGA""",10,3,3990,-2.5,-0.02439,-0.02439,12800.0,3680.0,1.0
10000,7952,1986-01-09,"""68391610""","""OMFGA""",10,3,3990,-2.5,0.0,0.0,1400.0,3680.0,1.0
10000,7952,1986-01-10,"""68391610""","""OMFGA""",10,3,3990,-2.5,0.0,0.0,8500.0,3680.0,1.0
10000,7952,1986-01-13,"""68391610""","""OMFGA""",10,3,3990,-2.625,0.05,0.05,5450.0,3680.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
93436,53453,2012-12-24,"""88160R10""","""TSLA""",11,3,9999,34.28,0.008235,0.008235,379504.0,113779.0,15.0
93436,53453,2012-12-26,"""88160R10""","""TSLA""",11,3,9999,33.59,-0.020128,-0.020128,614673.0,113779.0,15.0
93436,53453,2012-12-27,"""88160R10""","""TSLA""",11,3,9999,33.69,0.002977,0.002977,579663.0,113779.0,15.0


In [5]:
# Form BAB signal

# Log returns
df = df.with_columns(pl.col("retx").log1p().alias("logxret"))

# Make sure shrcd is 10 or 11
df = df.filter((df['shrcd'] == 10) | (df['shrcd'] == 11))

df

permno,permco,date,ncusip,ticker,shrcd,exchcd,siccd,prc,ret,retx,vol,shrout,cfacshr,logxret
i64,i64,date,str,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64
10000,7952,1986-01-07,"""68391610""","""OMFGA""",10,3,3990,-2.5625,null,null,1000.0,3680.0,1.0,null
10000,7952,1986-01-08,"""68391610""","""OMFGA""",10,3,3990,-2.5,-0.02439,-0.02439,12800.0,3680.0,1.0,-0.024692
10000,7952,1986-01-09,"""68391610""","""OMFGA""",10,3,3990,-2.5,0.0,0.0,1400.0,3680.0,1.0,0.0
10000,7952,1986-01-10,"""68391610""","""OMFGA""",10,3,3990,-2.5,0.0,0.0,8500.0,3680.0,1.0,0.0
10000,7952,1986-01-13,"""68391610""","""OMFGA""",10,3,3990,-2.625,0.05,0.05,5450.0,3680.0,1.0,0.04879
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93436,53453,2012-12-24,"""88160R10""","""TSLA""",11,3,9999,34.28,0.008235,0.008235,379504.0,113779.0,15.0,0.008201
93436,53453,2012-12-26,"""88160R10""","""TSLA""",11,3,9999,33.59,-0.020128,-0.020128,614673.0,113779.0,15.0,-0.020333
93436,53453,2012-12-27,"""88160R10""","""TSLA""",11,3,9999,33.69,0.002977,0.002977,579663.0,113779.0,15.0,0.002973
